# Machine Learning Foundation

## Section 2, Part a: Regression Intro: Transforming Target 

## Learning objectives

By the end of this lesson, you will be able to:

* Apply transformations to make target variable more normally distributed for regression
* Apply inverse transformations to be able to use these in a regression context

In [2]:
!pip install helper

  Created wheel for helper: filename=helper-2.5.0-py2.py3-none-any.whl size=19163 sha256=3c8d0829b4ab7d9bdf0f711bf4e7e7299745fe5ada8573e03735b154f36f2b2d
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\f2\c4\1a\a1c3faf3ffa2c4ef8106bf3ee4fd6ad0a7853d866a03bb9a89
Successfully built helper


In [4]:
!pip uninstall celery
!pip install celery==5.0.5

  Created wheel for click-didyoumean: filename=click_didyoumean-0.0.3-py3-none-any.whl size=2152 sha256=d5703222822e285ae75dd76c0a69a54641a12d57fb4d7a411c5affc1b887f86a
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\c2\97\1e\c7dda0b8e9616dcd8df2a2882e556ef62879967ac31e0e3d04
Successfully built click-didyoumean


In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from helper import (plot_exponential_data, 
                    plot_square_normal_data)

import warnings
warnings.simplefilter("ignore")

ModuleNotFoundError: No module named 'grp'

### Loading in Boston Data

**Note:** See `helper.py` file to see how boston data is read in from SciKit Learn.

In [8]:
from helper import boston_dataframe
boston_data = boston_dataframe()

ModuleNotFoundError: No module named 'grp'

In [ ]:
boston_data.head(15)

### Determining Normality

Making our target variable normally distributed often will lead to better results

If our target is not normally distributed, we can apply a transformation to it and then fit our regression to predict the transformed values.

How can we tell if our target is normally distributed? There are two ways:

* Visually
* Using a statistical test

#### Visually

Plotting a histogram:

In [ ]:
boston_data.MEDV.hist();

Does not look normal due to that right tail. Let's try to verify statistically:

In [ ]:
from scipy.stats.mstats import normaltest # D'Agostino K^2 Test

Without getting into Bayesian vs. frequentist debates, for the purposes of this lesson, the following will suffice:

* This is a statistical test that tests whether a distribution is normally distributed or not. It isn't perfect, but suffice it to say: 
    * This test outputs a "p-value". The _higher_ this p-value is the _closer_ the distribution is to normal.
    * Frequentist statisticians would say that you accept that the distribution is normal (more specifically: fail to reject the null hypothesis that it is normal) if p > 0.05.

In [ ]:
normaltest(boston_data.MEDV.values)

p-value _extremely_ low. Our y variable we've been dealing with this whole time was not normally distributed!

Linear Regression assumes a normally distributed residuals which can be aided by transforming y variable. Let's try some common transformations to try and get y to be normally distributed: 

* Log
* Square root
* Box cox

### Testing log

The log transform can transform data that is significantly skewed right to be more normally distributed:

In [ ]:
data = plot_exponential_data()

In [ ]:
plt.hist(np.log(data));

**Apply transform to Boston data:**

In [ ]:
log_medv = np.log(boston_data.MEDV)

In [ ]:
log_medv.hist();

In [ ]:
normaltest(log_medv)

Conclusion: closer, but still not normal.

### Exercise: 

The square root transformation is another transformation that can transform non-normally distributed data into normally distributed data:

In [ ]:
data = plot_square_normal_data()

Slightly skewed right.

In [ ]:
plt.hist(np.sqrt(data));

Apply the square root transformation to the Boston data target and test whether the result is normally distributed.

In [ ]:
pass # your code here

In [ ]:
# Instructor Solution

sqrt_medv = np.sqrt(boston_data.MEDV)
plt.hist(sqrt_medv)

In [ ]:
normaltest(sqrt_medv)

### Box cox

The box cox transformation is a parametrized transformation that tries to get distributions "as close to a normal distribution as possible".

It is defined as:

$$ \text{boxcox}(y_i) = \frac{y_i^{\lambda} - 1}{\lambda} $$

You can think of as a generalization of the square root function: the square root function uses the exponent of 0.5, but box cox lets its exponent vary so it can find the best one.

In [ ]:
from scipy.stats import boxcox

In [ ]:
bc_result = boxcox(boston_data.MEDV)
boxcox_medv = bc_result[0]
lam = bc_result[1]

In [ ]:
lam

In [ ]:
boston_data['MEDV'].hist();

In [ ]:
plt.hist(boxcox_medv);

In [ ]:
normaltest(boxcox_medv)

Significantly more normally distributed (according to p value) than the other two distributions - above 0.05, even!

Now that we have a normally distributed y-variable, let's try a regression!

### Testing regression:

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (StandardScaler, 
                                   PolynomialFeatures)

In [ ]:
lr = LinearRegression()

**Reload clean version of `boston_data`:**

In [ ]:
boston_data = boston_dataframe()

Same steps as before.

**Create X and y**

In [ ]:
y_col = "MEDV"

X = boston_data.drop(y_col, axis=1)
y = boston_data[y_col]

**Create Polynomial Features**

In [ ]:
pf = PolynomialFeatures(degree=2, include_bias=False)
X_pf = pf.fit_transform(X)

**Train test split**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_pf, y, test_size=0.3, 
                                                    random_state=72018)

**Fit `StandardScaler` on `X_train` as before**

In [ ]:
s = StandardScaler()
X_train_s = s.fit_transform(X_train)

**Discuss: what transformation do we need to apply next?**

Apply the appropriate transformation.

In [ ]:
pass # your code here

In [ ]:
# Instructor Solution
bc_result2 = boxcox(y_train)
y_train_bc = bc_result2[0]
lam2 = bc_result2[1]

As before, we'll now:

1. Fit regression
1. Transform testing data
1. Predict on testing data

In [ ]:
y_train_bc.shape

In [ ]:
lr.fit(X_train_s, y_train_bc)
X_test_s = s.transform(X_test)
y_pred_bc = lr.predict(X_test_s)

### Discussion

* Are we done?
* What did we predict?
* How would you interpret these predictions?

#### Inverse transform

Every transformation has an inverse transformation. The inverse transformation of $f(x) = \sqrt{x}$ is $f^{-1}(x) = x^2$, for example. Box cox has an inverse transformation as well: notice that we have to pass in the lambda value that we found from before:

In [ ]:
from scipy.special import inv_boxcox

In [ ]:
# code from above
bc_result = boxcox(boston_data.MEDV)
boxcox_medv = bc_result[0]
lam = bc_result[1]

In [ ]:
inv_boxcox(boxcox_medv, lam)[:10]

In [ ]:
boston_data['MEDV'].values[:10]

Exactly the same, as we would hope!

### Exercise:

1. Apply the appropriate inverse transformation to `y_pred_bc`.
2. Calculate the $R^2$ using the result of this inverse transformation and `y_test`.  

**Hint:** Should be two lines of code.

In [ ]:
pass # your code here

In [ ]:
# Instructor Solution
y_pred_tran = inv_boxcox(y_pred_bc,lam2)
r2_score(y_pred_tran,y_test)

## LAB Exercise: 

### Determine the R^2 of a LinearRegression without the box cox transformation. Is it higher or lower?

In [ ]:
### BEGIN SOLUTION
lr = LinearRegression()
lr.fit(X_train_s,y_train)
lr_pred = lr.predict(X_test_s)
r2_score(lr_pred,y_test)
### END SOLUTION

---
### Machine Learning Foundation (C) 2020 IBM Corporation